In [5]:
%pip install robin_stocks openpyxl pandas

Note: you may need to restart the kernel to use updated packages.


this is an example from this github: https://github.com/dwei-cn/python-tricks/blob/main/robinhood_hack.ipynb

TODO: limit to a date range to 

In [104]:
import robin_stocks.robinhood as r
import pandas as pd

email = input("Please enter your Robinhood email address: ")
login = r.login(email)

# Get all option orders
#all_orders = r.orders.get_all_option_orders()
option_trades = pd.DataFrame(r.get_all_option_orders())
option_trades = option_trades[~(
    option_trades['opening_strategy'].str.contains('spread', case=False, na=False) | 
    option_trades['closing_strategy'].str.contains('spread', case=False, na=False) |
    option_trades['opening_strategy'].str.contains('iron', case=False, na=False) | 
    option_trades['closing_strategy'].str.contains('iron', case=False, na=False)
)]

option_trades_filled = option_trades[[
    #'account_number', 
    'chain_symbol', 'state', 'created_at', 'quantity', 'direction', 'premium', 'opening_strategy', 'closing_strategy'
    ]]


option_trades_filled = option_trades_filled[option_trades_filled['state'] == 'filled']

# option_trades_filled
option_trades_filled['premium'] = pd.to_numeric(option_trades_filled['premium'], errors='coerce')
option_trades_filled['quantity'] = pd.to_numeric(option_trades_filled['quantity'], errors='coerce')

option_trades_filled['premium'].fillna(0)
option_trades_filled['quantity'].fillna(0)

option_trades_filled['profit'] = option_trades_filled.apply(lambda x: float(x['premium']) * float(x['quantity']) if x['direction'] == 'credit' else -1 * float(x['premium']) * float(x['quantity']),
                           axis = 1)

print(option_trades_filled)
     
print(option_trades_filled['profit'].sum())


Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
     chain_symbol   state                   created_at  quantity direction  \
0            UVXY  filled  2024-08-02T19:54:47.321337Z       1.0     debit   
1            SMCI  filled  2024-07-25T18:48:12.477405Z       1.0     debit   
2             SPY  filled  2024-07-22T19:30:17.299152Z       1.0    credit   
3             SPY  filled  2024-07-19T15:51:18.251325Z       1.0     debit   
4             IWM  filled  2024-07-19T13:41:54.847198Z       1.0    credit   
...           ...     ...                          ...       ...       ...   
1167         AAPL  filled  2019-04-24T19:12:53.837640Z       2.0    credit   
1168         AAPL  filled  2019-04-24T19:12:28.838015Z       2.0     debit   
1182          SPY  filled  2019-04-23T16:02:42.691669Z       2.0    credit   
1184          SPY  filled  2019-04-23T15:39:59.901637Z       2.0     debit   
1195         AMRS  fill

In [99]:
import robin_stocks.robinhood as r
import pandas as pd

email = input("Please enter your Robinhood email address: ")
login = r.login(email)

# Get all option orders
#all_orders = r.orders.get_all_option_orders()
option_trades = pd.DataFrame(r.get_all_option_orders())
option_trades = option_trades[option_trades['opening_strategy'].str.contains('spread', case=False, na=False) | option_trades['closing_strategy'].str.contains('spread', case=False, na=False)
       | option_trades['opening_strategy'].str.contains('iron', case=False, na=False) | option_trades['closing_strategy'].str.contains('iron', case=False, na=False)]

option_trades_filled = option_trades[[
    #'account_number', 
    'chain_symbol', 'state', 'created_at', 'quantity', 'direction', 'premium', 'opening_strategy', 'closing_strategy'
    ]]

option_trades_filled = option_trades_filled[option_trades_filled['state'] == 'filled'].sort_values(by = 'chain_symbol')

# Calculate 'profit'
option_trades_filled['profit'] = option_trades_filled.apply(
    lambda x: float(x['premium']) * float(x['quantity']) if x['direction'] == 'credit' else -1 * float(x['premium']) * float(x['quantity']),
    axis=1
)

# Display the DataFrame
option_trades_filled

# Sum of 'profit'
total_profit = option_trades_filled['profit'].sum()
print(total_profit)

Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
4655.0


In [3]:

r.export_completed_option_orders("./", "rh_options.csv")

Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...


In [12]:
import pandas as pd

# Load your spreadsheet
df = pd.read_csv('rh_options.csv')

# Convert 'order_created_at' to datetime
df['order_created_at'] = pd.to_datetime(df['order_created_at'])

# Filter the DataFrame to only include rows starting from March 1st
df = df[df['order_created_at'] >= '2024-01-01']

df['P&L'] = df.apply(lambda row: -row['price'] * row['processed_quantity'] if row['side'] == 'buy' else row['price'] * row['processed_quantity'], axis=1)

# Summarize P&L for each option symbol
summary = df.groupby('chain_symbol')['P&L'].sum().reset_index()

# Calculate total P&L
total_pnl = df['P&L'].sum()

# Print the summary
print(total_pnl)


-51.030000000000015


This next cell collects and groups spread orders

In [118]:
import pandas as pd
import numpy as np

# Load your spreadsheet
df = pd.read_csv('rh_options.csv')

# Convert 'order_created_at' to datetime
df['order_created_at'] = pd.to_datetime(df['order_created_at'])

# Filter the DataFrame to only include rows starting from March 1st

# Filter the DataFrame to include rows where either 'opening_strategy' or 'closing_strategy' contains 'spread'
df = df[df['opening_strategy'].str.contains('spread', case=False, na=False) | df['closing_strategy'].str.contains('spread', case=False, na=False)
   | df['opening_strategy'].str.contains('iron', case=False, na=False) | df['closing_strategy'].str.contains('iron', case=False, na=False)]

# Group by 'order_created_at' and 'opening_strategy'
grouped = df.groupby(['order_created_at'])

# Define a function to aggregate the strike prices
def aggregate_strike_prices(group):
    strikes = [f"{'+' if row['side'] == 'buy' else '-'}{row['strike_price']}" for _, row in group.iterrows()]
    return '/'.join(strikes)


# Apply the custom aggregation function
aggregated_df = grouped.apply(lambda group: pd.Series({
    'chain_symbol': group['chain_symbol'].iloc[0],
    'expiration_date': group['expiration_date'].iloc[0],
    'strike_price': aggregate_strike_prices(group),
    'option_type': group['option_type'].iloc[0],
    'side': group['side'].iloc[0],
    'direction': group['direction'].iloc[0],
    'order_quantity': group['order_quantity'].sum(),
    'order_type': group['order_type'].iloc[0],
    'opening_strategy': group['opening_strategy'].iloc[0],
    'closing_strategy': group['closing_strategy'].iloc[0],
    'price': group['price'].iloc[0],
    'processed_quantity': group['processed_quantity'].sum()
}), include_groups=False)

# Sort the DataFrame so the most recent date is at the top
#aggregated_df = aggregated_df.sort_values(by='order_created_at', ascending=False)

# Add a new column 'cost' which is the product of 'price' and 'processed_quantity'
aggregated_df['cost'] = aggregated_df['price'] * aggregated_df['processed_quantity'] * 100

aggregated_df['cost'] = aggregated_df.apply(lambda x: float(x['cost']) if x['direction'] == 'debit' else -1 * float(x['cost']),
    axis = 1)  
print(aggregated_df['cost'].sum())

aggregated_df.to_csv('rh_option_spreads.csv', index=False)

-10468.0


This cell collects all non spread orders

In [124]:
import pandas as pd
import numpy as np

# Load your spreadsheet
df = pd.read_csv('rh_options.csv')

# Convert 'order_created_at' to datetime
df['order_created_at'] = pd.to_datetime(df['order_created_at'])

# Filter the DataFrame to only include rows starting from March 1st

# Filter the DataFrame to include rows where either 'opening_strategy' or 'closing_strategy' contains 'spread'
df = df[~(df['opening_strategy'].str.contains('spread', case=False, na=False) | df['closing_strategy'].str.contains('spread', case=False, na=False)
          | df['opening_strategy'].str.contains('iron', case=False, na=False) | df['closing_strategy'].str.contains('iron', case=False, na=False))]

# Group by 'order_created_at' and 'opening_strategy'
grouped = df.groupby(['order_created_at'])

# Apply the custom aggregation function
aggregated_df = grouped.apply(lambda group: pd.Series({
    'chain_symbol': group['chain_symbol'].iloc[0],
    'expiration_date': group['expiration_date'].iloc[0],
    'strike_price': group['strike_price'].iloc[0],
    'option_type': group['option_type'].iloc[0],
    'side': group['side'].iloc[0],
    'direction': group['direction'].iloc[0],
    'order_quantity': group['order_quantity'].sum(),
    'order_type': group['order_type'].iloc[0],
    'opening_strategy': group['opening_strategy'].iloc[0],
    'closing_strategy': group['closing_strategy'].iloc[0],
    'price': group['price'].iloc[0],
    'processed_quantity': group['processed_quantity'].sum()
}), include_groups=False).reset_index()

# Sort the DataFrame so the most recent date is at the top
#aggregated_df = aggregated_df.sort_values(by='order_created_at', ascending=False)

# Add a new column 'cost' which is the product of 'price' and 'processed_quantity'
aggregated_df['cost'] = aggregated_df['price'] * aggregated_df['processed_quantity'] * 100

aggregated_df['cost'] = aggregated_df.apply(lambda x: float(x['cost']) if x['direction'] == 'credit' else -1 * float(x['cost']),
   axis = 1)  
print(aggregated_df['cost'].sum())

aggregated_df.to_csv('rh_option_not_spreads.csv', index=False)

-8556.0


In [68]:
import pandas as pd


# Load your spreadsheet
#df = pd.read_csv('option_orders_Mar-09-2024.csv')

all_orders = r.orders.get_all_option_orders()
df = pd.DataFrame(all_orders)

# Convert 'order_created_at' to datetime
df['created_at'] = pd.to_datetime(df['created_at'])

# Filter the DataFrame to only include rows starting from March 1st
#df = df[df['order_created_at'] >= '2019-02-12']

#df.loc[df['side'] == 'buy', 'strike_price'] = '+' + df['strike_price'].astype(str)
#df.loc[df['side'] == 'sell', 'strike_price'] = '-' + df['strike_price'].astype(str)

# Group the DataFrame by 'order_created_at'
# grouped = df.groupby('created_at')
# # Define a function to combine values with a '/' character if they are not all the same
# def combine_values(series):
#     unique_values = series.unique()
#     if len(unique_values) == 1:
#         return unique_values[0]
#     else:
#         return ' / '.join(unique_values)

# # Apply the function to all columns
# combined_df = grouped.agg(combine_values)

# Reset the index of the combined DataFrame
# combined_df.reset_index(inplace=True)
# combined_df.sort_values(by=['chain_symbol','order_created_at'], ascending=False, inplace=True)

# Save the combined DataFrame to a new CSV file
df.to_csv('combined.csv', index=False)

Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...


lets seperate trades before trying to parse them

In [4]:
import pandas as pd

# Load your spreadsheet
df = pd.read_csv('rh_options.csv')

# Convert 'order_created_at' to datetime
df['order_created_at'] = pd.to_datetime(df['order_created_at'])

# Filter the DataFrame to only include rows starting from March 1st
#df = df[df['order_created_at'] >= '2024-03-01']

#df['abs_strike_price'] = df['strike_price'].abs()

#df.loc[df['side'] == 'buy', 'strike_price'] = '+' + df['strike_price'].astype(str)
#df.loc[df['side'] == 'sell', 'strike_price'] = '-' + df['strike_price'].astype(str)

#Group the DataFrame by 'order_created_at'
grouped = df.groupby('order_created_at')
# Define a function to combine values with a '/' character if they are not all the same
def combine_values(series):
    if series.name in 'strike_price':
        unique_values = series.unique()
        if len(unique_values) == 1:
            return unique_values[0]
        else:
            unique_values = [float(i) for i in unique_values]
            # Sort the unique values in descending order before joining
            sorted_values = sorted(unique_values, key=abs, reverse=True)
            return ' / '.join(map(str, sorted_values))
    else:
        return series.iloc[0]

# Apply the function to all columns
combined_df = grouped.agg(combine_values)

# Reset the index of the combined DataFrame
combined_df.reset_index(inplace=True)
combined_df.sort_values(by=['chain_symbol'], inplace=True)

# Filter the DataFrame to only include rows where 'chain_symbol' is 'NVDA'
#filtered_df = combined_df[combined_df['chain_symbol'] == 'NVDA']

# Save the filtered DataFrame to a new CSV file
combined_df.to_csv('combined.csv', index=False)
